In [1]:
import polars as pl 
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from typing import Dict
from datetime import date
from sklearn.preprocessing import RobustScaler

In [2]:
from config import RUTA_RENTABILIDAD_PROCESADO, RUTA_DATA_INTERIM, RUTA_CLIENTES_CLUSTERIZADOS

In [3]:
RUTA_CLIENTES_PROCESADO = r"C:\Users\DF357JZ\EY\INTERBANCO - INTERCONSUMO - PRICING - Interbanco_-_Pricing\03. Relationship\2.1 Analytics\data\02_interim\\"+"clientes_procesado.parquet"

In [4]:
cluster = (
    pl.scan_parquet(RUTA_CLIENTES_CLUSTERIZADOS)
    .select(['AÑOMES_CORTE','ID_CLIENTE','CLUSTER'])
    )

Sectores que contienen el 80% de clientes

In [5]:
lista_sectores_relevantes = (
    pl.scan_parquet(RUTA_CLIENTES_PROCESADO)
    .unique('ID_CLIENTE')
    .group_by('SECTOR_ECONOMICO')
    .len()
    .sort('len',descending=True)
    .with_columns(acum = (pl.col('len')/pl.col('len').sum()))
    .with_columns(pl.col('acum').cum_sum())
    .filter(pl.col('acum')<0.8)
    .collect()
    .get_column('SECTOR_ECONOMICO')
    .to_list()
)

Base de clientes con variables sector y ubicacion

In [6]:
variables_socioeconomicas = (
    pl.scan_parquet(RUTA_CLIENTES_PROCESADO)
    .select(['FECHA','ID_CLIENTE','SECTOR_ECONOMICO','AGENCIA_PRINCIPAL'])
    .with_columns(pl.col('FECHA').dt.month_start())
    .with_columns(
        pl.when(
            pl.col('AGENCIA_PRINCIPAL').str.contains('METRO') | 
            pl.col('AGENCIA_PRINCIPAL').str.contains('METROPOLITANA') |
            pl.col('AGENCIA_PRINCIPAL').str.contains('PRINCIPAL') |
            pl.col('AGENCIA_PRINCIPAL').str.contains('Principal') |
            pl.col('AGENCIA_PRINCIPAL').str.contains('PYME') |
            pl.col('AGENCIA_PRINCIPAL').str.contains('Pyme')
        )
        .then(pl.lit('Metropolitana'))
        .otherwise(pl.lit('Rural'))
        .alias('AGENCIA_PRINCIPAL')
    )
    .with_columns(
        pl
        .when(pl.col('SECTOR_ECONOMICO').is_in(lista_sectores_relevantes))
        .then(pl.col('SECTOR_ECONOMICO'))
        .otherwise(pl.lit('OTRO'))
        .alias('SECTOR_ECONOMICO')
        )
    .rename({'FECHA':'AÑOMES_CORTE'})
    .filter(pl.col('AÑOMES_CORTE')>=date(2024,1,1))
)

unificamos variables de cliente

In [7]:
bases_cliente = (
    variables_socioeconomicas
    .join(cluster, on=['ID_CLIENTE','AÑOMES_CORTE'])
    .collect()
)

Preparamos la base de datos

In [8]:
df_rentabilidad = (
    pl.scan_parquet(RUTA_RENTABILIDAD_PROCESADO)
    .filter(pl.col('RUBRO').is_in(
        [
            'Costos Operacionales',
            'Comisiones por Negocios',
            'Sostenimiento',
            'Costos Transaccionales'
        ]
    ).not_()
    )
    .sort(['RUBRO'])
    .lazy()
)

In [9]:
df = (
    df_rentabilidad
    .group_by(['AÑOMES_CORTE','ID_CLIENTE','NRO_CUENTA','DESC_PRODUCTO','DESC_SUBPRODUCTO'])
    .agg(pl.col('MONTO').sum())
    .filter(pl.col('DESC_SUBPRODUCTO').is_not_null())
    .sort(['AÑOMES_CORTE','ID_CLIENTE','NRO_CUENTA','DESC_PRODUCTO','DESC_SUBPRODUCTO'])
    .lazy()
)

In [10]:
df_rentabilidad_promedio = (
    df
    .sort(['ID_CLIENTE','NRO_CUENTA','AÑOMES_CORTE'])
    .group_by(['ID_CLIENTE','NRO_CUENTA','DESC_PRODUCTO'])
    .agg(
        pl.col('MONTO').mean()
    )
    .collect()
)

In [52]:
df_rentabilidad_promedio.filter(
        pl.col('DESC_PRODUCTO')=='Disvisas'
        ).get_column(
            'MONTO'
            ).to_numpy()

array([1.45214331e+02, 1.19386043e+00, 1.85496300e+00, ...,
       1.39058293e+00, 2.15884477e+03, 1.10249685e+00])

In [11]:
# Dictionary to store KBinsDiscretizer for each product
discretizers = {}

In [13]:
# Create a copy of the datasets to avoid modifying the originals
train_data = df_rentabilidad_promedio.clone()

In [14]:
# Creamos lista de productos
lista_de_productos = train_data.get_column('DESC_PRODUCTO').unique().to_list()

In [12]:
def train_ratings(train_data):
    for product in lista_de_productos:
        product_data = train_data.filter(pl.col('DESC_PRODUCTO')==product)

        # Filter positive values
        positive_data = product_data.filter(pl.col('MONTO')>0).get_column('MONTO').to_numpy().reshape(-1,1)

        if positive_data.size > 30:

            discretizer = KBinsDiscretizer(n_bins=5,encode='ordinal',strategy='quantile')
            discretizer.fit(positive_data)
            discretizers[product] = discretizer

    return discretizers

In [15]:
def apply_ratings(discretizers: Dict, Producto: str, monto: float):

    if Producto in list(discretizers.keys()):
            
        if monto > 0:

            monto = np.array(monto).reshape(-1,1)

            rating = (discretizers[Producto].transform(monto)) + 2

        else:

            rating = 1
            
    else:

        rating = -1

    return rating

In [16]:
# Funcion para asignar ratings
discretizers = train_ratings(train_data=train_data)

In [17]:
train_data_discretize = (
    train_data
    .with_columns(
        pl.struct(['DESC_PRODUCTO','MONTO'])
        .map_elements(
            lambda x: apply_ratings(
                discretizers,
                Producto=x['DESC_PRODUCTO'],
                monto=x['MONTO'],
                ),
                return_dtype=pl.Float64,
        )
        .cast(int).alias('rating')
    )
)

In [18]:
df_rating = (
    train_data_discretize.drop('MONTO')
    .join(
        bases_cliente.filter(pl.col('AÑOMES_CORTE')==date(2024,8,1)).drop('AÑOMES_CORTE'),
        on=['ID_CLIENTE'],
        how='inner'
    )
    .select(['ID_CLIENTE','NRO_CUENTA','SECTOR_ECONOMICO','AGENCIA_PRINCIPAL','CLUSTER','DESC_PRODUCTO','rating'])
    .rename({'DESC_PRODUCTO':'PRODUCTO','AGENCIA_PRINCIPAL':'REGION'})
)

# Obtenemos matriz cliente producto

Obtenemos matriz de cliente producto con ratings

In [19]:
cliente_producto_ratings = (
    df_rating
    .pivot(index='ID_CLIENTE',columns='PRODUCTO',values='rating',aggregate_function='first')
)
cliente_producto_ratings.head()

ID_CLIENTE,Monetarios Plus Quetzales,Disvisas,Planes de Pago,Quetzales,Depósitos Monetarios Dólares,Ahorro Rendimiento Quetzales,Depósitos Monetarios Quetzales,Monetarios Preferencial Dólares,Doc. Descontados,Ahorro Corriente Quetzales,Cuentas Corrientes,Ahorro Rendimiento Dólares,InterDía Quetzales,Ahorro Euros,Ahorro Virtual Quetzales,InterAuto,InterSorteo Quetzales,InterFuturo Quetzales,Ahorro Corriente Dólares,Cobranza,Ahorro Preferencial Dólares,Visa Banco Internacional,InterDía Dólares,Plazo Fijo Quetzales,Inversión Creciente Quetzales,InterMoneda Quetzales,InterCasa,InterMoneda Dólares,Dólares,Financiamiento de Importación,Cuenta Concentradora Quetzales,InterPréstamo,Plazo Fijo Dólares,Sobregiros,Inversión Creciente Dólares,Cuenta InterNómina Quetzales,FHA,Cuenta Monedero Quetzales,Bonos Hipotecarios,Cartas de Crédito de Exportación,Cartas de Crédito de Importación,Cartas de Crédito Stand By
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""401695""",6,null,null,null,null,null,null,null,null,null,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""564160""",5,6,2,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""563528""",2,6,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""226274""",4,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""408694""",5,6,null,null,null,null,null,null,null,4,null,null,null,null,null,null,null,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


Guardamos orden de columnas

In [20]:
columnas_productos = cliente_producto_ratings.columns[1:]

obtenemos informacion promedio de rating pro cluster

In [27]:
cluster_producto_rating = (
    df_rating
    .pivot(
        index=['SECTOR_ECONOMICO','REGION','CLUSTER'],
        columns='PRODUCTO',
        values='rating',
        aggregate_function='mean'
    )
    .select(['SECTOR_ECONOMICO','REGION','CLUSTER']+columnas_productos)
)

cluster_producto_rating.head()

SECTOR_ECONOMICO,REGION,CLUSTER,Monetarios Plus Quetzales,Disvisas,Planes de Pago,Quetzales,Depósitos Monetarios Dólares,Ahorro Rendimiento Quetzales,Depósitos Monetarios Quetzales,Monetarios Preferencial Dólares,Doc. Descontados,Ahorro Corriente Quetzales,Cuentas Corrientes,Ahorro Rendimiento Dólares,InterDía Quetzales,Ahorro Euros,Ahorro Virtual Quetzales,InterAuto,InterSorteo Quetzales,InterFuturo Quetzales,Ahorro Corriente Dólares,Cobranza,Ahorro Preferencial Dólares,Visa Banco Internacional,InterDía Dólares,Plazo Fijo Quetzales,Inversión Creciente Quetzales,InterMoneda Quetzales,InterCasa,InterMoneda Dólares,Dólares,Financiamiento de Importación,Cuenta Concentradora Quetzales,InterPréstamo,Plazo Fijo Dólares,Sobregiros,Inversión Creciente Dólares,Cuenta InterNómina Quetzales,FHA,Cuenta Monedero Quetzales,Bonos Hipotecarios,Cartas de Crédito de Exportación,Cartas de Crédito de Importación,Cartas de Crédito Stand By
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""COMERCIO INTERNO (GUATEMALA)_-…","""Rural""","""Potenciales""",3.28169,4.571429,5.021277,-1.0,4.5,4.666667,3.0,3.25,2.714286,null,3.384615,null,3.0,null,1.0,4.6,4.0,6.0,2.0,null,null,3.0,null,null,null,null,null,null,-1.0,null,null,6.0,null,-1.0,null,null,null,null,null,null,null,null
"""COMERCIO INTERNO (GUATEMALA)_-…","""Rural""","""Lovers Activos""",3.933333,4.330275,3.0746,-1.0,4.083333,3.75,4.533333,5.285714,3.0,3.909091,3.612903,4.5,4.214286,-1.0,4.5,4.064516,3.7,3.0,3.333333,null,3.0,1.642857,4.0,3.666667,5.5,4.333333,4.75,null,-1.0,5.5,null,5.833333,null,-1.0,null,null,null,null,null,null,null,null
"""COMBUSTIBLES PARA VEHICULO_-""","""Rural""","""Lovers Activos""",4.205128,5.0,1.871409,-1.0,5.0,4.428571,3.285714,null,null,null,4.0,null,null,null,null,5.0,null,null,null,null,null,null,null,null,null,null,null,null,-1.0,null,null,null,null,null,null,null,null,null,null,null,null,null
"""ENTIDADES SIN FINES DE LUCRO-""","""Rural""","""Ahorrador Potencial""",4.490741,4.90099,null,-1.0,4.733333,4.3125,5.186441,4.545455,null,5.571429,null,5.5,4.636364,null,4.222222,null,null,5.0,null,null,null,null,5.75,3.263158,4.625,2.0,null,3.526316,null,null,-1.0,null,null,null,3.0,null,null,null,null,null,null,null
"""COMERCIO INTERNO (GUATEMALA)_-…","""Rural""","""Ahorrador Potencial""",5.160535,4.702532,null,-1.0,4.6,5.272727,5.342466,4.964286,null,5.333333,1.0,3.75,5.129032,-1.0,4.222222,null,null,null,4.333333,null,5.0,null,5.285714,2.25,3.0,4.666667,null,4.0,-1.0,null,-1.0,null,null,null,3.666667,null,null,-1.0,null,null,-1.0,null


In [22]:
informacion_cliente = df_rating.drop(['NRO_CUENTA','rating'])

In [189]:
cliente_producto_ratings.write_parquet(r"99. NBO\data\interim\cliente_producto_ratings.parquet")
cluster_producto_rating.write_parquet(r"99. NBO\data\interim\cluster_producto_rating.parquet")
informacion_cliente.write_parquet(r"99. NBO\data\interim\informacion_cliente.parquet")

In [25]:
cliente_producto_ratings.fill_null(0).write_parquet(r"99. NBO\data\interim\cliente_rating.parquet")